# QP-based trajectory optimisation

In this second tutorial on trajectory optimisation, we are going to optimise the trajectory for a robot manipulator similarly to what has been done in the lab. I decided to add an extra exercise just to make sure everyone had understood the process for the coming exam.




In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
Let us import the standard libraries for the tutorials and load a new robot

In [2]:
# %load tp3/generated/inverse_kinematics_import
import pinocchio as pin
import numpy as np
import time
from numpy.linalg import pinv,inv,norm,svd,eig
from utils.load_ur5_with_obstacles import load_ur5_with_obstacles,Target # helper function to load scene
import matplotlib.pylab as plt
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
import unittest


Let's load the complete UR5 model with obstacles

In [12]:
robot = load_ur5_with_obstacles(reduced=False, obstacles = False)



In [13]:
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


Let us define two configurations start and goal, and find a trajectory between both such that
* the trajectory is continuous and differentiable
* the joint limits are respected

In [14]:
q_start = np.array([-0.62831853, -0.62831853, -0.31415927, -0.62831853, -0.62831853,
       -0.62831853])

q_end = - q_start
viz.display(q_start)


In [15]:
viz.display(q_start)

robot.model.lowerPositionLimit = q_start.copy()
robot.model.upperPositionLimit = q_end.copy()


Let us first see that a simple linear interpolation resolves our problem

In [16]:
def jointlimitsviolated(robot,q):
    up = max(q - robot.model.upperPositionLimit)
    down = max(robot.model.lowerPositionLimit - q)
    '''Return true if config not in joint limits'''
    return max(0,max(up,down)) > 0.

def interpolate(q_start, q_end, t):
    assert (0. <= t and t <= 1)
    return q_start + (q_end - q_start) * t

nbconfigs = 1000
inc = 1. / nbconfigs
for i in range(nbconfigs):
    q = interpolate(q_start, q_end, i*inc)
    viz.display(q)
    if jointlimitsviolated(robot, q):
        print("joint limits violation at configuration ", i)

no violation. But what if we want to include a cost?

## Setting up the trajectory formulation


We will formulate our optimisation problem using a single degree 3 polynomial to represent our trajectory.
As we have seen before we could use a variety of curves to represent this, each with its specific pros and cons.
In this example we will use a hermite cubic spline that has the following formulation on a [0,1] interval:

$$ \boldsymbol{p}(t) = \left(2t^3 - 3t^2 + 1\right) \boldsymbol{p}_0 + \left(t^3 - 2t^2 + t\right) \boldsymbol{m}_0 + \left(-2t^3 + 3t^2\right) \boldsymbol{p}_1 + \left(t^3 - t^2\right) \boldsymbol{m}_1 $$

where $\boldsymbol{p}_0 $ and $\boldsymbol{p}_1$ are the initial and final control points and $\boldsymbol{m}_0 $ and $\boldsymbol{m}_1$ are the starting and ending tangent (ie velocity).

We can observe that if t is fixed the expression of the polynomial is linear in the control variables $\boldsymbol{m}_i $ and $\boldsymbol{p}_i$. 

With this formulation we can see that in our case the $\boldsymbol{p}_i$ are constant, and only the initial velocities are variables of our problem. Indeed $\boldsymbol{p}_0 = q_{start}$ and $\boldsymbol{p}_1 = q_{end}$ .

This also gives us the dimension of our control points, ie 6.
As a result, we have 12 variables to consider in our optimisation problem, 6 for $\boldsymbol{m}_0 $ and 6 more for $\boldsymbol{m}_1$.

### discretising the curve
To verify our constraints, we decide to only verify that they are satisfied at discrete points along the trajectory.
For instance, we can decide to discretise the trajectory into N points. Each of these points only depends on the control points of the spline, since t is fixed. This will thus give us a linear expression of any point as a function of $\boldsymbol{m}_0 $ and $\boldsymbol{m}_1 $, which we can write in matrix form:




$$ \boldsymbol{p}_{i} = \underbrace{\left(t_i^3 - 2_i^2 + t_i\right)}_{\boldsymbol{d}_i} \boldsymbol{m}_0 +  \underbrace{\left(t_i^3 - t_i^2\right)}_{\boldsymbol{e}_i} \boldsymbol{m}_1 + \underbrace{\left(2t_i^3 - 3t_i^2 + 1\right) \boldsymbol{p}_0 + \left(-2t_i^3 + 3t_i^2\right) \boldsymbol{p}_1}_{\boldsymbol{b}_i}$$

defining $\mathbf{x} = [\mathbf{m}_0; \mathbf{m}_1]^T $ we can then write in matrix form:

$$ \boldsymbol{p}_{i} = diag(\underbrace{\begin{bmatrix}
\mathbf{d}_i & \mathbf{d}_i &\mathbf{d}_i & \boldsymbol{e}_i  & \boldsymbol{e}_i & \boldsymbol{e}_i
\end{bmatrix}}_{\mathbf{A}_i}) \boldsymbol{x}+ \boldsymbol{b}_i $$

Let us write the function that computes the matrices $\mathbf{A}_i$ and $\mathbf{b}_i$:

In [17]:
def pi(p0, p1, ti):
    ti2 = ti *ti
    ti3 = ti2*ti
    di = ti3 - 2* ti2 + ti
    ei = ti3 - ti2
    bi = (2*ti3 - 3* ti2 + 1) * p0 + (3*ti2 - 2*ti3) * p1
    Ai = np.diag(np.array([di,di,di,ei,ei,ei]))
    return (Ai, bi)


## Dealing with joint constraints

To find a trajectory that respects joint limits, we will check that all discretised point satisfy joint limits.
For each $\mathbf{p}_i$, we need to verify that $\mathbf{p}_i \leq \mathbf{q}^+$ and $\mathbf{q}^- \leq \mathbf{p}_i$.

This means that we must verify 

$$\mathbf{A}_i \mathbf{x} + \mathbf{b}_i \leq \mathbf{q}^+ $$, ie 
$$\mathbf{A}_i \mathbf{x} \leq \mathbf{q}^+ - \mathbf{b}_i  $$

Similarly we write
$$\mathbf{A}_i \mathbf{x} + \mathbf{b}_i \geq \mathbf{q}^-$$, which gives

$$\mathbf{b}_i - \mathbf{q}^- \geq -\mathbf{A}_i \mathbf{x} $$

Getting q_min and q_max is easy:

In [18]:
qmax =  robot.model.upperPositionLimit
qmin =  robot.model.lowerPositionLimit
nsteps = 3 # check constraints 48 times between start and end

We can thus easily define the matrix inequality to satisfy joint limits at position i:

In [19]:
def jointlimitconstraint(p0, p1, ti):
    (Ai, bi) = pi(p0, p1, ti)
    bmin = bi - qmin
    Amin = -Ai
    bmax = qmax - bi
    Amax = Ai.copy()
    #now stacking the constraints
    A = np.vstack([Amin,Amax])
    b = np.concatenate((bmin,bmax))
    return (A,b)

A0, b0 = jointlimitconstraint(q_start, q_end, 0.5)

We can now stack all constraints into one big matrix and vector that will give us the inequalities for all discrete points and solve our optimisation problem using quadprog (for an introduction to quadprog take a look [here](https://scaron.info/blog/quadratic-programming-in-python.html) )

In [30]:
A = np.zeros((0,6))#empty matrix
b = np.zeros(0) #empty vector

for i in range(1, nsteps):
    ti = 1. / nsteps * i
    Ai, bi = jointlimitconstraint(q_start, q_end, ti)
    A = np.vstack([A,Ai])
    b = np.concatenate((b,bi))
   
from quadprog import solve_qp

#solves a feasibility qp given a constraint matrix A <= b 
def solve_qp_feasibility(A,b):
    #create a simple cost as we are only looking for a feasible solution for now
    G = np.eye(6); a = np.zeros(6)
    return solve_qp(G,a,-A.T,-b)

res = solve_qp_feasibility(A,b)


def plot_from_res(x):
    for i in range(1000):
        Ai, bi = pi(q_start, q_end, 1./1000*i)
        p = Ai.dot(x) + bi
        viz.display(p)

plot_from_res(res[0])
        

The solution we obtained is such that m0 and m1 are zeros. You can write the code that computes the corresponding curve and displays it to verify that this corresponds to the linear interpolation. So apparently these constraints were not useful! Let's consider the same problem with a cost function that maximises uniformly the start and end velocities along the positive axis:

In [40]:
def solve_qp_max_angle_values(A,b):
    #create a simple cost as we are only looking for a feasible solution for now
    G = np.eye(6)*0.001; a = -np.ones(6) * 100
    return solve_qp(G,a,-A.T,-b)

res2 = solve_qp_max_angle_values(A,b)
plot_from_res(res2[0])

### Additional considerations

This was a rather naive cost function. Write one that minimises the sum of all joint angles for the 3rd joint. You can achieve this using the pi function again and integrating it in your sum.